In [2]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered


In [3]:
import pandas as pd
import nilearn
import numpy as np
import glob 
import os
import pickle

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf


from pathlib import Path


from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

import seaborn as sns


from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import fdr, threshold

%matplotlib inline


/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
def onetoughjar(path2dic):
    with open(path2dic, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)

In [90]:
def interactor(m):
    X['int'] = X['demo_comb_income_v2']*X['%s'%m]
    # Fit the multinomial logistic regression model
    logit_model = sm.Logit(y, X)
    result = logit_model.fit()
    return(result)



In [91]:
savedDict = onetoughjar('/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/tmp/SSBABCDanalysis_11-23-2024_07-36-16')

In [92]:
savedDict.keys()

dict_keys(['df', 'X', 'y', 'X_train', 'X_test', 'y_train', 'y_test', 'best_pipeline', 'X_reg', 'X_trainReg', 'X_testReg', 'y_trainReg', 'y_testReg', 'logistic'])

In [96]:
savedDict['logistic'].params

const                             -0.060342
rsfmri_c_ngd_ad_ngd_cgc            2.349196
rsfmri_c_ngd_ad_ngd_vs             1.014924
rsfmri_c_ngd_cgc_ngd_cgc          -4.294523
rsfmri_c_ngd_smh_ngd_smm           2.050590
rsfmri_c_ngd_sa_ngd_vta            2.046234
rsfmri_cor_ngd_au_scs_aalh        -4.775641
rsfmri_cor_ngd_au_scs_ptrh         1.873602
rsfmri_cor_ngd_au_scs_vtdcrh       3.165791
rsfmri_cor_ngd_cerc_scs_cdelh      4.400505
rsfmri_cor_ngd_cerc_scs_ptlh      -1.572657
rsfmri_cor_ngd_cerc_scs_aglh      -4.761894
rsfmri_cor_ngd_cerc_scs_vtdclh    -1.157367
rsfmri_cor_ngd_cerc_scs_hprh       5.637630
rsfmri_cor_ngd_cerc_scs_agrh      -2.247335
rsfmri_cor_ngd_cerc_scs_vtdcrh    -6.258692
rsfmri_cor_ngd_copa_scs_thplh      1.195095
rsfmri_cor_ngd_copa_scs_vtdclh    12.925653
rsfmri_cor_ngd_copa_scs_plrh      -3.023761
rsfmri_cor_ngd_df_scs_ptlh         2.173067
rsfmri_cor_ngd_df_scs_pllh         0.883310
rsfmri_cor_ngd_df_scs_aalh         4.021799
rsfmri_cor_ngd_none_scs_ptlh    

In [34]:
colz = savedDict['X_reg']

In [39]:
list(colz.columns[1:])

['rsfmri_c_ngd_ad_ngd_cgc',
 'rsfmri_c_ngd_ad_ngd_vs',
 'rsfmri_c_ngd_cgc_ngd_cgc',
 'rsfmri_c_ngd_smh_ngd_smm',
 'rsfmri_c_ngd_sa_ngd_vta',
 'rsfmri_cor_ngd_au_scs_aalh',
 'rsfmri_cor_ngd_au_scs_ptrh',
 'rsfmri_cor_ngd_au_scs_vtdcrh',
 'rsfmri_cor_ngd_cerc_scs_cdelh',
 'rsfmri_cor_ngd_cerc_scs_ptlh',
 'rsfmri_cor_ngd_cerc_scs_aglh',
 'rsfmri_cor_ngd_cerc_scs_vtdclh',
 'rsfmri_cor_ngd_cerc_scs_hprh',
 'rsfmri_cor_ngd_cerc_scs_agrh',
 'rsfmri_cor_ngd_cerc_scs_vtdcrh',
 'rsfmri_cor_ngd_copa_scs_thplh',
 'rsfmri_cor_ngd_copa_scs_vtdclh',
 'rsfmri_cor_ngd_copa_scs_plrh',
 'rsfmri_cor_ngd_df_scs_ptlh',
 'rsfmri_cor_ngd_df_scs_pllh',
 'rsfmri_cor_ngd_df_scs_aalh',
 'rsfmri_cor_ngd_none_scs_ptlh',
 'rsfmri_cor_ngd_none_scs_thprh',
 'rsfmri_cor_ngd_none_scs_cderh',
 'rsfmri_cor_ngd_rst_scs_bs',
 'rsfmri_cor_ngd_rst_scs_thprh',
 'rsfmri_cor_ngd_rst_scs_hprh',
 'rsfmri_cor_ngd_rst_scs_vtdcrh',
 'rsfmri_cor_ngd_smh_scs_crcxlh',
 'rsfmri_cor_ngd_smh_scs_pllh',
 'rsfmri_cor_ngd_smh_scs_cderh',
 'rs

In [36]:
X = savedDict['X']

In [41]:
X = X[list(colz.columns[1:])]

In [42]:
X.shape

(3700, 49)

In [43]:
y = savedDict['y']

In [44]:
y.shape

(3700,)

In [50]:
brain = ['rsfmri_c_ngd_ad_ngd_cgc', 'rsfmri_c_ngd_ad_ngd_vs',
       'rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_c_ngd_smh_ngd_smm',
       'rsfmri_c_ngd_sa_ngd_vta', 'rsfmri_cor_ngd_au_scs_aalh',
       'rsfmri_cor_ngd_au_scs_ptrh', 'rsfmri_cor_ngd_au_scs_vtdcrh',
       'rsfmri_cor_ngd_cerc_scs_cdelh', 'rsfmri_cor_ngd_cerc_scs_ptlh',
       'rsfmri_cor_ngd_cerc_scs_aglh', 'rsfmri_cor_ngd_cerc_scs_vtdclh',
       'rsfmri_cor_ngd_cerc_scs_hprh', 'rsfmri_cor_ngd_cerc_scs_agrh',
       'rsfmri_cor_ngd_cerc_scs_vtdcrh', 'rsfmri_cor_ngd_copa_scs_thplh',
       'rsfmri_cor_ngd_copa_scs_vtdclh', 'rsfmri_cor_ngd_copa_scs_plrh',
       'rsfmri_cor_ngd_df_scs_ptlh', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_df_scs_aalh', 'rsfmri_cor_ngd_none_scs_ptlh',
       'rsfmri_cor_ngd_none_scs_thprh', 'rsfmri_cor_ngd_none_scs_cderh',
       'rsfmri_cor_ngd_rst_scs_bs', 'rsfmri_cor_ngd_rst_scs_thprh',
       'rsfmri_cor_ngd_rst_scs_hprh', 'rsfmri_cor_ngd_rst_scs_vtdcrh',
       'rsfmri_cor_ngd_smh_scs_crcxlh', 'rsfmri_cor_ngd_smh_scs_pllh',
       'rsfmri_cor_ngd_smh_scs_cderh', 'rsfmri_cor_ngd_smh_scs_ptrh',
       'rsfmri_cor_ngd_smh_scs_hprh', 'rsfmri_cor_ngd_smh_scs_aarh',
       'rsfmri_cor_ngd_smm_scs_hplh', 'rsfmri_cor_ngd_smm_scs_aglh',
       'rsfmri_cor_ngd_smm_scs_thprh', 'rsfmri_cor_ngd_sa_scs_crcxlh',
       'rsfmri_cor_ngd_sa_scs_thplh', 'rsfmri_cor_ngd_vta_scs_hplh',
       'rsfmri_cor_ngd_vs_scs_hplh', 'rsfmri_cor_ngd_vs_scs_thprh',
       'rsfmri_cor_ngd_vs_scs_plrh', 'rsfmri_cor_ngd_vs_scs_hprh', 'interview_age',
       'bmi_perc', 'sex_M']

In [51]:
pvalues = {}

In [52]:
for ROI in brain:
    try:
        pvalues[ROI] = interactor(ROI)
    except:
        pvalues[ROI] = 'error'
    

/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_96824/2375773377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['int'] = X['demo_comb_income_v2']*X['%s'%m]


Optimization terminated successfully.
         Current function value: 0.529505
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529721
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.528425
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529654
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529367
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529756
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529686
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529038
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529746
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529613
  

In [54]:
intP = []
for ps in pvalues.values():
    intP.append(ps.pvalues[-1])

In [55]:
# Create a Series without a custom index
seriesP = pd.Series(intP)

# Assign a custom index
seriesP.index = pvalues.keys()
# Replace 'r' with 1
seriesP = seriesP.replace('r', 1)

# Replace NaN with 1
seriesP = seriesP.fillna(1)

# Display the Series
print(seriesP)

rsfmri_c_ngd_ad_ngd_cgc           1.474841e-01
rsfmri_c_ngd_ad_ngd_vs            5.530529e-01
rsfmri_c_ngd_cgc_ngd_cgc          1.351834e-03
rsfmri_c_ngd_smh_ngd_smm          3.635078e-01
rsfmri_c_ngd_sa_ngd_vta           8.454847e-02
rsfmri_cor_ngd_au_scs_aalh        7.707925e-01
rsfmri_cor_ngd_au_scs_ptrh        4.358952e-01
rsfmri_cor_ngd_au_scs_vtdcrh      2.165500e-02
rsfmri_cor_ngd_cerc_scs_cdelh     6.889941e-01
rsfmri_cor_ngd_cerc_scs_ptlh      2.884348e-01
rsfmri_cor_ngd_cerc_scs_aglh      2.464524e-01
rsfmri_cor_ngd_cerc_scs_vtdclh    3.236377e-01
rsfmri_cor_ngd_cerc_scs_hprh      1.503627e-01
rsfmri_cor_ngd_cerc_scs_agrh      4.486701e-02
rsfmri_cor_ngd_cerc_scs_vtdcrh    1.291085e-02
rsfmri_cor_ngd_copa_scs_thplh     5.055608e-02
rsfmri_cor_ngd_copa_scs_vtdclh    1.378073e-01
rsfmri_cor_ngd_copa_scs_plrh      7.520731e-03
rsfmri_cor_ngd_df_scs_ptlh        6.555527e-01
rsfmri_cor_ngd_df_scs_pllh        2.659025e-03
rsfmri_cor_ngd_df_scs_aalh        8.005686e-02
rsfmri_cor_ng

In [56]:
# Perform FDR correction
rejected, pvals_corrected, _, _ = multipletests(seriesP, alpha=0.05, method='fdr_bh')

# Print the results
print("Rejected (Significant after FDR):", rejected)
print("Corrected p-values:", pvals_corrected)


Rejected (Significant after FDR): [False False  True False False False False False False False False False
 False False False False False False False  True False False False  True
 False False False False False False False False False False False False
 False False  True False False False False False  True  True False]
Corrected p-values: [2.82681848e-01 6.49837194e-01 1.64032213e-02 5.17723217e-01
 1.98688899e-01 8.05049896e-01 5.39133545e-01 8.56449736e-02
 7.71017192e-01 4.51881141e-01 4.13688010e-01 4.75342888e-01
 2.82681848e-01 1.40583297e-01 6.74233278e-02 1.48508499e-01
 2.81606245e-01 5.04963364e-02 7.51487232e-01 2.49948378e-02
 1.98688899e-01 2.57308600e-01 6.40097702e-01 1.64032213e-02
 5.36032152e-01 5.39133545e-01 5.39133545e-01 5.26199849e-01
 3.21332933e-01 2.20344098e-01 4.51881141e-01 1.98688899e-01
 1.40583297e-01 8.96809241e-02 4.13688010e-01 8.56449736e-02
 8.56449736e-02 9.76947681e-01 3.25289190e-02 7.93552166e-01
 4.57180065e-01 7.82909685e-01 8.62489302e-01 1.6

In [57]:
# Create a Series without a custom index
sigP = pd.Series(pvals_corrected)
# Assign a custom index
sigP.index = seriesP.index


In [59]:
# Filter the series using the boolean list
filtered_values = sigP[rejected]

In [75]:
filtered_values.index

Index(['rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_sa_scs_thplh',
       'interview_age', 'bmi_perc'],
      dtype='object')

# Summary
Household income interacts with:
* rsfmri_c_ngd_cgc_ngd_cgc         1.640322e-02 Average correlation between cingulo-opercular network and cingulo-opercular network
* rsfmri_cor_ngd_df_scs_pllh       2.499484e-02 Average correlation between default network and ASEG ROI left-pallidum
* rsfmri_cor_ngd_none_scs_cderh    1.640322e-02 Average correlation between none network and ASEG ROI right-caudate
* rsfmri_cor_ngd_sa_scs_thplh      3.252892e-02 Average correlation between salience network and ASEG ROI left-thalamus-proper
* interview_age                    1.548630e-06
* bmi_perc                         1.044164e-13

# Probing the interaction

In [71]:
result = pvalues['rsfmri_c_ngd_cgc_ngd_cgc']

In [97]:
def tableMaker(result):
    # Assuming `result` is your BinaryResultsWrapper object
    # Extract coefficients, confidence intervals, and compute odds ratios
    params = result.params
    conf = result.conf_int()
    conf['OR'] = np.exp(params)  # Compute odds ratios
    conf.columns = ['2.5%', '97.5%', 'OR']  # Rename columns

    # Compute odds ratios for confidence intervals
    conf['2.5%'] = np.exp(conf['2.5%'])
    conf['97.5%'] = np.exp(conf['97.5%'])

    # Format as a DataFrame for publication
    table = conf[['OR', '2.5%', '97.5%']]

    # Optionally, round the table for readability
    table = table.round(2)
    return(table)

    

In [98]:
tableMaker(savedDict['logistic'])

,OR,2.5%,97.5%
const,0.94,0.51,1.730000e+00
rsfmri_c_ngd_ad_ngd_cgc,10.48,0.00,2.811608e+06
rsfmri_c_ngd_ad_ngd_vs,2.76,0.00,8.387370e+03
rsfmri_c_ngd_cgc_ngd_cgc,0.01,0.00,4.758000e+01
rsfmri_c_ngd_smh_ngd_smm,7.77,0.01,4.198340e+03
rsfmri_c_ngd_sa_ngd_vta,7.74,0.01,8.055660e+03
rsfmri_cor_ngd_au_scs_aalh,0.01,0.00,1.184000e+01
rsfmri_cor_ngd_au_scs_ptrh,6.51,0.00,1.217898e+04
rsfmri_cor_ngd_au_scs_vtdcrh,23.71,0.01,8.152372e+04
rsfmri_cor_ngd_cerc_scs_cdelh,81.49,0.02,4.222093e+05


In [76]:
tableHolder = {}

In [77]:
for ROI in list(filtered_values.index):
    result = pvalues['%s'%ROI]
    tableHolder['%s'%ROI] = tableMaker(result)

In [101]:
tableHolder.keys()

dict_keys(['rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_cor_ngd_df_scs_pllh', 'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_sa_scs_thplh', 'interview_age', 'bmi_perc'])

In [107]:
tableHolder['bmi_perc']

,OR,2.5%,97.5%
rsfmri_c_ngd_ad_ngd_cgc,0.88,0.11,7.02
rsfmri_c_ngd_ad_ngd_vs,3.34,0.83,13.54
rsfmri_c_ngd_cgc_ngd_cgc,1.09,0.24,4.84
rsfmri_c_ngd_smh_ngd_smm,2.32,0.77,6.96
rsfmri_c_ngd_sa_ngd_vta,1.12,0.29,4.27
rsfmri_cor_ngd_au_scs_aalh,0.14,0.04,0.53
rsfmri_cor_ngd_au_scs_ptrh,1.44,0.36,5.75
rsfmri_cor_ngd_au_scs_vtdcrh,0.74,0.18,3.10
rsfmri_cor_ngd_cerc_scs_cdelh,0.07,0.01,0.35
rsfmri_cor_ngd_cerc_scs_ptlh,0.93,0.30,2.89


In [61]:
savedDict.update({'sigP':sigP})

In [79]:
savedDict.update({'odds':tableHolder})

In [81]:
savedDict['odds']

{'rsfmri_c_ngd_cgc_ngd_cgc':                                    OR  2.5%  97.5%
 rsfmri_c_ngd_ad_ngd_cgc          0.82  0.11   6.39
 rsfmri_c_ngd_ad_ngd_vs           3.20  0.80  12.84
 rsfmri_c_ngd_cgc_ngd_cgc         0.50  0.11   2.30
 rsfmri_c_ngd_smh_ngd_smm         2.30  0.78   6.83
 rsfmri_c_ngd_sa_ngd_vta          1.08  0.29   4.07
 rsfmri_cor_ngd_au_scs_aalh       0.14  0.04   0.53
 rsfmri_cor_ngd_au_scs_ptrh       1.56  0.40   6.15
 rsfmri_cor_ngd_au_scs_vtdcrh     0.85  0.20   3.52
 rsfmri_cor_ngd_cerc_scs_cdelh    0.07  0.01   0.34
 rsfmri_cor_ngd_cerc_scs_ptlh     0.93  0.31   2.86
 rsfmri_cor_ngd_cerc_scs_aglh     0.71  0.10   4.99
 rsfmri_cor_ngd_cerc_scs_vtdclh   0.74  0.38   1.43
 rsfmri_cor_ngd_cerc_scs_hprh     1.42  0.20   9.80
 rsfmri_cor_ngd_cerc_scs_agrh     1.55  0.42   5.77
 rsfmri_cor_ngd_cerc_scs_vtdcrh   0.57  0.11   2.97
 rsfmri_cor_ngd_copa_scs_thplh    1.34  0.22   7.97
 rsfmri_cor_ngd_copa_scs_vtdclh   4.08  0.56  29.87
 rsfmri_cor_ngd_copa_scs_plrh     1.

In [82]:
import pickle
from datetime import datetime
from time import time


In [83]:
datefmt='%m-%d-%Y_%I-%M-%S'

In [84]:
def adillyofapickle(basepath,dic, name):
    st = datetime.fromtimestamp(time()).strftime(datefmt)
    if os.path.exists(os.path.join(basepath,'tmp')):
        print('already have tmp')
    else:
        os.makedirs(os.path.join(basepath,'tmp'))
    pickle.dump(dic, open(os.path.join(basepath,'tmp','%s_%s'%(name,st)), 'wb'), protocol=4)

In [85]:
basepath = '/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024'




In [86]:
adillyofapickle(basepath,savedDict, 'SSBABCDanalysisInt')

already have tmp


## Probe the interaction for plots
Didn't have the dataset with factor level of CHI, loaded that and re-merged with original

In [5]:
dataPath = '/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/tmp'
savedDict = onetoughjar(os.path.join(dataPath,'SSBABCDanalysisInt_11-26-2024_10-44-32'))

In [27]:


factorDF = pd.read_csv('/Users/gracer/Library/CloudStorage/OneDrive-UniversityofWyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/ABCD/Yana_SSIB_2024/data/justCHI.csv')

In [51]:
df = pd.merge(factorDF, savedDict['df'], on='subjectkey', how='outer')

In [52]:
df.shape

(3701, 442)

In [54]:
df = df.dropna(subset=['demo_comb_income_factor'])

In [55]:
df

,subjectkey,demo_comb_income_factor,demo_comb_income_v2,bkfs_soda,interview_age,anthroheightcalc,rsfmri_c_ngd_ad_ngd_ad,rsfmri_c_ngd_ad_ngd_cgc,rsfmri_c_ngd_ad_ngd_ca,rsfmri_c_ngd_ad_ngd_dt,...,race_7,race_8,race_9,race_10,race_11,race_12,race_13,race_14,race_15,race_16
0,NDAR_INVF0C17HWX,9.0,1.134865,2.263514,2.208108,-0.351927,0.068443,-0.013937,0.043602,-0.052279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NDAR_INV72J2TKB2,9.0,1.134865,-1.736486,-3.791892,-1.296927,0.010384,0.009812,0.017131,0.052089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NDAR_INVXVM1T4AG,1.0,-6.865135,1.263514,2.208108,-5.226927,-0.021120,-0.001985,0.016845,0.000079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NDAR_INVTU1DE4PM,8.0,0.134865,1.263514,-0.791892,3.273073,0.039870,-0.018612,-0.093783,-0.059711,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NDAR_INVY43C1PAY,4.0,-3.865135,3.263514,-0.791892,1.773073,0.148359,-0.003445,-0.130025,-0.040669,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,NDAR_INV1MF1L9LR,8.0,0.134865,3.263514,6.208108,-4.101927,0.007844,-0.088140,-0.095973,0.034362,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3696,NDAR_INV9A6HTXF6,9.0,1.134865,-1.736486,11.208108,1.273073,0.128205,-0.015683,-0.118861,-0.017324,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3697,NDAR_INVEXWHLUB5,7.0,-0.865135,3.263514,-4.791892,-4.351927,0.044244,0.011726,-0.013017,-0.013231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3698,NDAR_INVWA4ACPCR,10.0,2.134865,1.263514,12.208108,-0.226927,-0.002446,0.036297,-0.096342,-0.014220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
nonBrain = df[['demo_comb_income_factor','interview_age', 'bmi_perc',  'sex_M','ssb_group',]]

In [72]:
ROIs = df.filter(like='rsfmri_', axis=1)

In [73]:
ROIs.shape

(3700, 416)

In [74]:
df.shape

(3700, 442)

In [75]:
Xfact = ROIs.join(nonBrain)

In [76]:
Xfact

,rsfmri_c_ngd_ad_ngd_ad,rsfmri_c_ngd_ad_ngd_cgc,rsfmri_c_ngd_ad_ngd_ca,rsfmri_c_ngd_ad_ngd_dt,rsfmri_c_ngd_ad_ngd_dla,rsfmri_c_ngd_ad_ngd_fo,rsfmri_c_ngd_ad_ngd_n,rsfmri_c_ngd_ad_ngd_rspltp,rsfmri_c_ngd_ad_ngd_smh,rsfmri_c_ngd_ad_ngd_smm,...,rsfmri_cor_ngd_vs_scs_plrh,rsfmri_cor_ngd_vs_scs_hprh,rsfmri_cor_ngd_vs_scs_agrh,rsfmri_cor_ngd_vs_scs_aarh,rsfmri_cor_ngd_vs_scs_vtdcrh,demo_comb_income_factor,interview_age,bmi_perc,sex_M,ssb_group
0,0.068443,-0.013937,0.043602,-0.052279,-0.001817,-0.073673,-0.008185,0.021034,-0.039437,0.198793,...,0.040415,0.109031,0.019496,-0.017181,0.022865,9.0,2.208108,-0.074089,1.0,1.0
1,0.010384,0.009812,0.017131,0.052089,0.004709,0.022979,0.018181,-0.012840,0.002369,-0.048014,...,0.009324,-0.005636,-0.112208,0.003579,0.045996,9.0,-3.791892,-0.166220,0.0,0.0
2,-0.021120,-0.001985,0.016845,0.000079,0.027425,0.056818,0.013597,0.030836,-0.049539,-0.069404,...,0.249363,0.062374,-0.213170,-0.070342,-0.078987,1.0,2.208108,-0.117158,1.0,1.0
3,0.039870,-0.018612,-0.093783,-0.059711,0.048004,-0.092068,-0.027847,0.014588,0.092321,0.066388,...,-0.011558,0.176237,-0.036178,-0.078277,0.198341,8.0,-0.791892,0.181997,0.0,1.0
4,0.148359,-0.003445,-0.130025,-0.040669,0.029820,-0.017150,-0.017234,-0.076365,0.117753,0.174538,...,-0.057407,0.012681,-0.040334,-0.115159,0.039500,4.0,-0.791892,0.132754,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,0.007844,-0.088140,-0.095973,0.034362,0.003104,0.034102,-0.004585,-0.045583,0.064742,-0.021000,...,-0.023009,0.020711,-0.015846,-0.025092,-0.018759,8.0,6.208108,-0.089739,1.0,1.0
3696,0.128205,-0.015683,-0.118861,-0.017324,0.022211,-0.030273,-0.014853,0.120055,0.142242,0.055583,...,-0.060750,-0.115547,0.241700,0.019459,-0.016186,9.0,11.208108,-0.092898,0.0,0.0
3697,0.044244,0.011726,-0.013017,-0.013231,-0.020813,-0.040568,0.010807,0.032868,0.029834,-0.001331,...,-0.030467,0.036232,-0.013794,0.044985,0.061419,7.0,-4.791892,0.209150,0.0,1.0
3698,-0.002446,0.036297,-0.096342,-0.014220,0.010990,0.027282,-0.038254,0.047381,-0.011857,-0.019707,...,0.063622,0.018049,0.026047,0.075362,0.082922,10.0,12.208108,0.009343,1.0,1.0


In [77]:
CHIlevels = [1,2,3,4,5,6,7,8,9,10]

In [78]:
breakOut = {}

In [79]:
for i in CHIlevels:
    print(i)
    subset_df_ = Xfact[Xfact['demo_comb_income_factor'] == i]
    subset_df_ = subset_df_.drop('demo_comb_income_factor', axis=1)
    breakOut[i] = subset_df_


1
2
3
4
5
6
7
8
9
10


In [80]:
breakOut[1].columns

Index(['rsfmri_c_ngd_ad_ngd_ad', 'rsfmri_c_ngd_ad_ngd_cgc',
       'rsfmri_c_ngd_ad_ngd_ca', 'rsfmri_c_ngd_ad_ngd_dt',
       'rsfmri_c_ngd_ad_ngd_dla', 'rsfmri_c_ngd_ad_ngd_fo',
       'rsfmri_c_ngd_ad_ngd_n', 'rsfmri_c_ngd_ad_ngd_rspltp',
       'rsfmri_c_ngd_ad_ngd_smh', 'rsfmri_c_ngd_ad_ngd_smm',
       ...
       'rsfmri_cor_ngd_vs_scs_ptrh', 'rsfmri_cor_ngd_vs_scs_plrh',
       'rsfmri_cor_ngd_vs_scs_hprh', 'rsfmri_cor_ngd_vs_scs_agrh',
       'rsfmri_cor_ngd_vs_scs_aarh', 'rsfmri_cor_ngd_vs_scs_vtdcrh',
       'interview_age', 'bmi_perc', 'sex_M', 'ssb_group'],
      dtype='object', length=420)

In [81]:
testers = ['rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_sa_scs_thplh',
       'interview_age', 'bmi_perc']

In [99]:
X = breakOut[8].drop('ssb_group', axis = 1)

In [97]:
y = breakOut[8]['ssb_group']

In [94]:
X.shape

(68, 419)

In [98]:
y.value_counts()

0.0    356
1.0    265
Name: ssb_group, dtype: int64

In [89]:
def modelerTest(i):
    X = breakOut[i].drop('ssb_group', axis = 1)
    y = breakOut[i]['ssb_group']
    logit_model = sm.Logit(y, X)
    result = logit_model.fit()
    return(result)

In [100]:
modelerTest(8)

/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


         Current function value: inf
         Iterations: 35


/Users/gracer/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [101]:
savedDict.keys()

dict_keys(['df', 'X', 'y', 'X_train', 'X_test', 'y_train', 'y_test', 'best_pipeline', 'X_reg', 'X_trainReg', 'X_testReg', 'y_trainReg', 'y_testReg', 'logistic', 'sigP', 'odds'])

In [103]:
savedDict['X_reg'].columns

Index(['const', 'rsfmri_c_ngd_ad_ngd_cgc', 'rsfmri_c_ngd_ad_ngd_vs',
       'rsfmri_c_ngd_cgc_ngd_cgc', 'rsfmri_c_ngd_smh_ngd_smm',
       'rsfmri_c_ngd_sa_ngd_vta', 'rsfmri_cor_ngd_au_scs_aalh',
       'rsfmri_cor_ngd_au_scs_ptrh', 'rsfmri_cor_ngd_au_scs_vtdcrh',
       'rsfmri_cor_ngd_cerc_scs_cdelh', 'rsfmri_cor_ngd_cerc_scs_ptlh',
       'rsfmri_cor_ngd_cerc_scs_aglh', 'rsfmri_cor_ngd_cerc_scs_vtdclh',
       'rsfmri_cor_ngd_cerc_scs_hprh', 'rsfmri_cor_ngd_cerc_scs_agrh',
       'rsfmri_cor_ngd_cerc_scs_vtdcrh', 'rsfmri_cor_ngd_copa_scs_thplh',
       'rsfmri_cor_ngd_copa_scs_vtdclh', 'rsfmri_cor_ngd_copa_scs_plrh',
       'rsfmri_cor_ngd_df_scs_ptlh', 'rsfmri_cor_ngd_df_scs_pllh',
       'rsfmri_cor_ngd_df_scs_aalh', 'rsfmri_cor_ngd_none_scs_ptlh',
       'rsfmri_cor_ngd_none_scs_thprh', 'rsfmri_cor_ngd_none_scs_cderh',
       'rsfmri_cor_ngd_rst_scs_bs', 'rsfmri_cor_ngd_rst_scs_thprh',
       'rsfmri_cor_ngd_rst_scs_hprh', 'rsfmri_cor_ngd_rst_scs_vtdcrh',
       'rsfmri_cor_ngd_s